In [2]:
pip install torch transformers datasets gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 70.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import Dataset
import gradio as gr

data = [
    {"text": "Riddle: What number becomes zero when you subtract 15 from half of it?\nAnswer: 30"},
    {"text": "Riddle: I am a number that when doubled and then reduced by 20 gives 40.\nAnswer: 30"},
    {"text": "Riddle: If you add 10 to a number and then subtract 5, you get 25.\nAnswer: 20"},
    {"text": "Riddle: I am 15 less than twice my value.\nAnswer: 15"},
    {"text": "Riddle: A number when halved and then increased by 10 becomes 25.\nAnswer: 30"},
    {"text": "Riddle: When you multiply a number by 3 and subtract 9, the result is 18.\nAnswer: 9"},
    {"text": "Riddle: If a number is decreased by 8 and then doubled, you get 14.\nAnswer: 15"},
    {"text": "Riddle: A number when tripled and then increased by 5 equals 20.\nAnswer: 5"},
    {"text": "Riddle: When you add 7 to half of a number, you get 19.\nAnswer: 24"},
    {"text": "Riddle: A number is increased by 9 and then halved to get 15.\nAnswer: 21"},
    {"text": "Riddle: When you subtract 4 from a number and then multiply by 3, the result is 33.\nAnswer: 15"},
    {"text": "Riddle: A number reduced by 6 equals one-third of itself.\nAnswer: 9"},
    {"text": "Riddle: When you double a number and add 10, you get 30.\nAnswer: 10"},
    {"text": "Riddle: A number, when 5 is subtracted and then multiplied by 2, gives 20.\nAnswer: 15"},
    {"text": "Riddle: If a number is multiplied by 4 and then decreased by 8, the result is 24.\nAnswer: 8"},
    {"text": "Riddle: A number, when divided by 2 and then increased by 7, equals 17.\nAnswer: 20"},
    {"text": "Riddle: When you subtract 3 from a number and then square the result, you get 49.\nAnswer: 10"},
    {"text": "Riddle: If 12 is added to a number, the result is three times the number.\nAnswer: 6"},
    {"text": "Riddle: A number increased by 50% equals 27.\nAnswer: 18"},
    {"text": "Riddle: If a number is halved and then 4 is subtracted, the result is 8.\nAnswer: 24"},
    {"text": "Riddle: A number, when 2 is added, becomes twice the original number.\nAnswer: 2"},
    {"text": "Riddle: When you triple a number and subtract 7, the result is 14.\nAnswer: 7"},
    {"text": "Riddle: A number, when reduced by 2 and then divided by 4, gives 5.\nAnswer: 22"},
    {"text": "Riddle: When you add 8 to a number and then multiply by 2, you get 40.\nAnswer: 12"},
    {"text": "Riddle: A number, when doubled, is 16 more than the number itself.\nAnswer: 16"},
    {"text": "Riddle: A number that is increased by 3 and then multiplied by 2 equals 26.\nAnswer: 10"},
    {"text": "Riddle: A number when reduced by 4 and then doubled equals 12.\nAnswer: 10"},
    {"text": "Riddle: If you subtract 2 from a number and then double it, you get 14.\nAnswer: 9"},
    {"text": "Riddle: A number when tripled and decreased by 5 equals 16.\nAnswer: 7"},
    {"text": "Riddle: If you add 5 to a number and then double the result, you get 30.\nAnswer: 10"}
]

dataset = Dataset.from_list(data)

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=128)

tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(["text"])
tokenized_dataset.set_format("torch")

model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Adjusted training arguments for better learning
training_args = TrainingArguments(
    output_dir="./gpt2-math-riddle",
    overwrite_output_dir=True,
    num_train_epochs=25,                    # Increased epochs for better convergence
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    learning_rate=2e-5,                     # Slightly lower learning rate for stability
    weight_decay=0.01,
    warmup_steps=50,                        # Reduced warmup steps
    save_steps=500,
    save_total_limit=2,
    logging_steps=50,
    prediction_loss_only=True,
    report_to=[]
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
)

trainer.train()

model.eval()
model.config.pad_token_id = tokenizer.eos_token_id

# Generate riddle with improved control
def generate_riddle(prompt):
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        input_ids,
        max_length=50,
        do_sample=False,                    # Switch to beam search for precision
        num_beams=5,                        # Use beam search for better coherence
        repetition_penalty=1.2,
        no_repeat_ngram_size=3,
        max_new_tokens=20,                  # Limit new tokens to control answer length
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id
    )

    # Decode the output
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Clean up output without overriding model’s answer
    if "\nAnswer:" in generated_text:
        parts = generated_text.split("\nAnswer:")
        riddle = parts[0].strip()
        if not riddle.startswith("Riddle:"):
            riddle = f"Riddle: {riddle}"
        # Extract answer, take first numeric token
        answer_part = parts[1].strip()
        answer = "".join(filter(str.isdigit, answer_part.split()[0]))  # First numeric token
        if not answer:  # If no number found
            answer = "10"  # Minimal fallback, but aim is to avoid this
        formatted_output = f"{riddle}\nAnswer: {answer}"
    else:
        riddle = generated_text.strip()
        if not riddle.startswith("Riddle:"):
            riddle = f"Riddle: {riddle}"
        formatted_output = f"{riddle}\nAnswer: 10"  # Minimal fallback

    return formatted_output

# Gradio interface
iface = gr.Interface(
    fn=generate_riddle,
    inputs="text",
    outputs="text",
    title="Math Riddle Generator",
    description="Enter a prompt to generate a single math riddle with its answer."
)

iface.launch()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
50,2.983300
100,1.172100
150,0.882800


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://fad0fb68baa0a2691d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
